## Saved work in progress code snippets

In [1]:
from Gaussain_Dushin_modules import *

RDKit WARNING: [12:02:37] Enabling RDKit 2019.09.3 jupyter extensions


#### From trying to run with checkpoint file

In [ ]:
def generate_Gjf_checkpoint(mol_charge_state, Excited_state, output_folder = os.getcwd()+"/", 
                          functional_basisset='b3lyp/6-311g(d,p)', number_of_procc='32', memory='30GB'):
    # generate gjf for running theodore to identify CT states
    # Will only run TDDFT

    #write the gaussian file

    
    a_str_gaussian="0 1\n"
    a_str_gaussian=mol_charge_state + " theodore\n\n"+a_str_gaussian
    
    a_str_gaussian="#p nosymm Geom=Checkpoint pop=full iop(9/40=3) td(NStates=20)  "+functional_basisset+"\n\n"+a_str_gaussian
        
    a_str_gaussian="%nprocshared="+number_of_procc+"\n\n"+a_str_gaussian
    a_str_gaussian="%mem="+memory+"\n"+a_str_gaussian
    #a_str_gaussian="%chk="+mol_charge_state+".chk  \n"+a_str_gaussian
    
    a_str_gaussian="%chk="+mol_charge_state+"_theodore.chk\n"+a_str_gaussian
    a_str_gaussian="%oldchk="+mol_charge_state+".chk\n"+a_str_gaussian
    a_str_gaussian=a_str_gaussian+"\n"

    #print(a_str_gaussian)
    text_file = open(output_folder+mol_charge_state+".gjf", "wt")
    n = text_file.write(a_str_gaussian)
    text_file.close()
    return mol_charge_state


def copy_checkpoint(hostname, username, password, remote_folder, remote_checkpoint_folder, mol_charge_state):


    with paramiko.SSHClient() as client:
        client.load_system_host_keys()
        client.set_missing_host_key_policy(paramiko.WarningPolicy)
        client.connect(hostname, username=username,password=password)

        stdin, stdout, stderr = client.exec_command("mkdir -p "+remote_checkpoint_folder+" \n"+
                                                    "cd "+remote_folder+" \n "+
                                                    "cp "+mol_charge_state+".chk "+remote_checkpoint_folder+" \n"+
                                                    "cd "+remote_checkpoint_folder+" \n "+
                                                    "chmod +rwx "+mol_charge_state+".chk \n"
                                                   )
                                                    
        print("Standard output:")
        print(stdout.read())
        print("Standard error:")
        print(stderr.read())
        print("Exit status: {}".format(stdout.channel.recv_exit_status()))
        client.close()
        

In [ ]:
#running job using checkpoint file

status = "Started"
print("Status = ", status)

#conversion from smiles to gjf

#for mol_num in range(0,len(mol_list)):
for mol_num in [1]:
    

    for state_num in range(0,len(states)):
        charge_state = "C" + str(charges[state_num]) + "S" + str(states[state_num])
        mol_charge_state =  mol_list_name+"_"+str(mol_num)+"_"+charge_state
        
        #copy chk file into checkpoint so not be get overwritten
        #copy_checkpoint(hostname, username, password, remote_folder, remote_checkpoint_folder, mol_charge_state)
        
        #writing gjf file based on .chk file
        checkpoint_gjf = generate_Gjf_checkpoint(mol_charge_state, states[state_num])
        generate_shfile_gaussian(walltime='7:59:01', memory="50gb", ncpus=60, timeout="8h", molnumber= mol_num,
                             Excited_state = states[state_num], mollist_name= mol_list, gjf_file = checkpoint_gjf)
        run_job_HPC(hostname = hostname,username = username, password = password, remotefolder = remote_checkpoint_folder,\
                             molnumber = mol_num, Excited_state = states[state_num],mollist_name= mol_list_name, gjf_file = checkpoint_gjf)

            
        print("Queued job for: ", mol_charge_state)
        os.remove(mol_charge_state+".gjf")
        os.remove(mol_charge_state+".sh")
        
print("All Jobs uploaded")  